In [1]:
# temporary solution to crashing
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import mysql.connector as connector
from pathlib import Path

import logging

In [3]:
home = os.path.expanduser('~')
os.chdir(home) # b/c we will be using universal paths

host = '127.0.0.1'
user = 'root' # change to your username
password = 'vasya_is_best_cat_12345' # change to your password
database = 'ai_proj_2025' # for the sake of sanity, we should all have this as the db name 

try:
    conn = connector.connect(
        host = host, 
        user = user, 
        password = password, 
        database = database
    )
    print('success')
except connector.Error as err:
    print(err)

success


In [4]:
research_dir = Path(home, 'Desktop', 'Education', 'Spring 2025', 'AI', 'research')
os.chdir(research_dir)

from data_helper import SQLDataset_Informative

os.chdir(home)

In [5]:
#Define the convnet (Eventually replae with mobile 2)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3) #specify the size (outer numbers) and amount (middle number) of filters
        self.pool = nn.MaxPool2d(2, 2) #specify pool size first number is size of pool, second is step size
        self.conv2 = nn.Conv2d(16, 8, 3) #new depth is amount of filters in previous conv layer
        self.fc1 = nn.Linear(54*54*8, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 3) #finial fc layer needs 19 outputs because we have 19 layers # ???

    def forward(self, x):
         # dont include label
        # print(f'x.shape before forward: {x.shape}')

        x = F.relu(self.conv1(x))
        # print(f'x.shape after conv1: {x.shape}')
        x = self.pool(x)
        # print(f'x.shape after pool #1: {x.shape}')

        x = F.relu(self.conv2(x))
        # print(f'x.shape after conv2: {x.shape}')
        x = self.pool(x)
        # print(f'x.shape after pool #2: {x.shape}')

        x = x.view(-1, 54*54*8) # flatten
        # print(f'x.shape after x.view: {x.shape}')

        x = F.relu(self.fc1(x))    #fully connected, relu   
        # print(f'x.shape after fc1: {x.shape}')        
        x = F.relu(self.fc2(x))    #fully connected, relu 
        # print(f'x.shape after fc2: {x.shape}')           
        x = self.fc3(x)            #output 
        # print(f'x.shape after fc3: {x.shape}')          
        return x

In [6]:
from torchvision.transforms import v2

# transforms
transformations = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True), 
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=Path('OneDrive - Stephen F. Austin State University', 'CrisisMMD_v2.0','CrisisMMD_v2.0'))

In [8]:
from torch.utils.data import DataLoader
train_loader = DataLoader(set, batch_size=32)

In [9]:
model = ConvNet()

In [10]:
test_result = model.forward(set.__getitem__(0))
test_result.shape

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor)!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor)!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)


In [11]:
learning_rate = .001
num_epochs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
from tqdm import tqdm 

#Train the model
model = ConvNet().to(device)

cross_entropy = nn.CrossEntropyLoss() #cross entropy loss for our loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #stocastic gradient descent for our optimization algorithm

for epoch in tqdm(range(num_epochs)):
    print('Epoch', epoch)
    for _, item in tqdm(enumerate(train_loader), total= len(train_loader), desc="Processing data in epoch {epoch}"):
        # print(f'item: ', item)
        images = item[0]
        # print('images', images.shape)
        labels = item[1]
        assert(labels.shape[0] == images.shape[0])
        # origin shape: 
        # input_layer:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model.forward(images)
        # print('outputs', outputs.shape)
        # print(labels.shape)
        loss = cross_entropy(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad() #apply optimization func
        loss.backward() #backpropagation
        optimizer.step() #single step updating parameters
        
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0


  0%|          | 0/1 [00:35<?, ?it/s]


KeyboardInterrupt: 